In [442]:
!ls    #to display the available files

ADTest.csv	 sample_submission.csv	Tester.csv
audio_test	 Tes.csv		test_post_competition.csv
audio_test.zip	 Test2.csv		Testv.csv
audio_train	 Test3.csv		train.csv
audio_train.zip  test.csv		Trainp.csv
csvexample.csv	 Test.csv		train_post_competition.csv
datalab		 testdocs.csv
MTest.csv	 Tester1.csv


In [0]:
file = open('train.csv','r')     #load csv files for manipulating train.csv files
words2=set()
j=0
for i in file:
  if(j!=0):
    #print(i)
    ls=i.split(",")[1]
    words2.add(ls)
  j=j+1
#print(words)
#print(len(words))
all_words2=sorted(list(words2))

In [0]:
vocab={ all_words2[j]:j for j in range(0,len(all_words2)) }   #for constructing vocabulary

In [382]:
print(vocab)

{'Acoustic_guitar': 0, 'Applause': 1, 'Bark': 2, 'Bass_drum': 3, 'Burping_or_eructation': 4, 'Bus': 5, 'Cello': 6, 'Chime': 7, 'Clarinet': 8, 'Computer_keyboard': 9, 'Cough': 10, 'Cowbell': 11, 'Double_bass': 12, 'Drawer_open_or_close': 13, 'Electric_piano': 14, 'Fart': 15, 'Finger_snapping': 16, 'Fireworks': 17, 'Flute': 18, 'Glockenspiel': 19, 'Gong': 20, 'Gunshot_or_gunfire': 21, 'Harmonica': 22, 'Hi-hat': 23, 'Keys_jangling': 24, 'Knock': 25, 'Laughter': 26, 'Meow': 27, 'Microwave_oven': 28, 'Oboe': 29, 'Saxophone': 30, 'Scissors': 31, 'Shatter': 32, 'Snare_drum': 33, 'Squeak': 34, 'Tambourine': 35, 'Tearing': 36, 'Telephone': 37, 'Trumpet': 38, 'Violin_or_fiddle': 39, 'Writing': 40}


The following are the test post competition files label extraction for private and public files.

In [422]:
file2 = open('test_post_competition.csv','r')
private=[]
public=[]
plabel=[]
pulabel=[]
iter1=0
for iter in file2:
  if(iter1==0):
    iter1=iter1+1
    print("in")
    continue
  #print(iter.split(",")[0])
  lname=iter.split(",")[1]
  name=iter.split(",")[0]
  usage=iter.split(",")[2]
  if(lname!="None"):
    #print(lname)
    temp=vocab[lname]
    #teslabel.append(np.eye(41)[vocab[lname]])
    if(usage=='Private'):
      private.append(name)
      plabel.append(np.eye(41)[vocab[lname]])
    if(usage=="Public"):
      public.append(name)
      pulabel.append(np.eye(41)[vocab[lname]])

in


In [424]:
plabel=np.array(plabel)
pulabel=np.array(pulabel)
print(len(private))
print(len(public))
print(plabel.shape)
print(pulabel.shape)

1299
301
(1299, 41)
(301, 41)


In [432]:
print(len(public))

301


In [395]:
file2 = open('test_post_competition.csv','r')   #code to load entire test file
tester=[]
teslabel=[]
iter1=0
for iter in file2:
  if(iter1==0):
    iter1=iter1+1
    print("in")
    continue
  #print(iter.split(",")[0])
  lname=iter.split(",")[1]
  name=iter.split(",")[0]
  if(lname!="None"):
    #print(lname)
    temp=vocab[lname]
    teslabel.append(np.eye(41)[vocab[lname]])
    tester.append(name)


in


In [0]:
tlabel=np.array(teslabel)

In [402]:
print(tester[0])

00326aa9.wav


Following is the class implementation to preprocess the test files.

In [0]:
#lister=dict()
class Collector3:
  
  def __init__(self,pth,duration,tester):
    

    self.myPath=pth
    self.iter=0
    self.liter=0
    self.duration=duration
    self.input_length=44100*self.duration
    self.lister=dict()
    self.tester=tester
  def pad(self,a):
    if(a.shape[1]>1400):
        return a[:,0:1400]
    else:
        return np.pad(a,[(0,0),(0,(1400-a.shape[1]))],'constant',constant_values=(0))

    
  def next_batch(self,it):
    onlyfiles = [f for f in sorted(listdir(self.myPath)) if isfile(join(self.myPath,f))]
    j=it
    max=0
    voc=dict()
    trainer=[]
    k=self.iter
    for i in self.tester:
        audio_path=self.myPath+"/"+i
        #print(i)
        data,sr=librosa.load(audio_path, sr=44100)
        #print(y.shape)
        #data,sr=librosa.load(audio_path, sr=44100)
        
        # Random offset / Padding
        if len(data) > self.input_length:
            max_offset = len(data) - self.input_length
            #print(max_offset)
            offset = np.random.randint(max_offset)
            #print(offset)
            data = data[offset:(self.input_length+offset)]
        else:
            if self.input_length > len(data):
                max_offset = self.input_length - len(data)
                offset = np.random.randint(max_offset)
            else:
                offset = 0
            data = np.pad(data, (offset, self.input_length - len(data) - offset), "constant")
        #S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
        #log_S = librosa.power_to_db(S, ref=np.max)
        mfcc=librosa.feature.mfcc(y=data, sr=sr, n_mfcc=40)
        #S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
        #log_S = librosa.power_to_db(S, ref=np.max)
        
        #mfcc        = librosa.feature.mfcc(S=log_S, n_mfcc=13)

        # Let's pad on the first and second deltas while we're at it
        #delta_mfcc  = librosa.feature.delta(mfcc)
        #delta2_mfcc = librosa.feature.delta(mfcc, order=2)
        #print(log_S.shape)
        trainer.append(mfcc)
     
    return np.array(trainer)
        

In [0]:
coll1=Collector3('audio_test',2,private) #preprocess private files with audio duration=2

In [0]:
Private_Test=coll1.next_batch(1299)

In [0]:
coll2=Collector3('audio_test',2,public) #preprocess public files with audio duration=2

In [0]:
Public_Test=coll2.next_batch(1209)

In [439]:
print(len(plabel))

1299


In [0]:
X_test=Collector3('audio_test',2,tester)

In [0]:
Test=X_test.next_batch(1600)

In [0]:
Test=np.expand_dims(Test,axis=-1)

In [411]:
print(Test.shape)
print(tlabel.shape)

(1600, 40, 173, 1)
(1600, 41)


Following code is to establish connection with the google drive

In [300]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [3]:
!kaggle competitions download -c freesound-audio-tagging   #keras command to download the competition file

  0%|                                                | 0.00/449k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 449k/449k [00:00<00:00, 73.9MB/s]
  0%|                                                | 0.00/440k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 440k/440k [00:00<00:00, 88.8MB/s]
  0%|                                                | 0.00/321k [00:00<?, ?B/s]
100%|█████████████████████████████████████████| 321k/321k [00:00<00:00, 104MB/s]
  0%|                                                | 0.00/228k [00:00<?, ?B/s]
100%|█████████████████████████████████████████| 228k/228k [00:00<00:00, 150MB/s]
100%|██████████████████████████████████████▉| 3.26G/3.27G [00:23<00:00, 155MB/s]
100%|███████████████████████████████████████| 3.27G/3.27G [00:23<00:00, 149MB/s]
100%|██████████████████████████████████████▉| 3.79G/3.79G [00:27<00:00, 108MB/s]
100%|███████████████████████████████████████| 3.79G/3.79G [00:27<00:00, 150MB/s]


In [4]:
!ls

audio_test.zip	 sample_submission.csv	    train_post_competition.csv
audio_train.zip  test_post_competition.csv
datalab		 train.csv


In [0]:
import zipfile
zip_ref = zipfile.ZipFile('audio_test.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [0]:
import zipfile
zip_ref = zipfile.ZipFile('audio_train.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [7]:
!ls

audio_test	audio_train.zip        test_post_competition.csv
audio_test.zip	datalab		       train.csv
audio_train	sample_submission.csv  train_post_competition.csv


In [8]:
!pip install librosa

    100% |████████████████████████████████| 1.6MB 8.3MB/s 
    100% |████████████████████████████████| 276kB 11.7MB/s 
    100% |████████████████████████████████| 327kB 11.0MB/s 
    100% |████████████████████████████████| 1.9MB 9.9MB/s 
    100% |████████████████████████████████| 15.9MB 2.0MB/s 
  Running setup.py bdist_wheel for librosa ... - \ | done
  Stored in directory: /content/.cache/pip/wheels/18/b8/10/f0f8f6ac60668a5cd75596cf14c25bb6b3ea1ecd815f058b7e
  Running setup.py bdist_wheel for audioread ... - done
  Stored in directory: /content/.cache/pip/wheels/53/02/90/7b5c4081b7470c550ab605f600bad237dde12a6b8999b11f50
  Running setup.py bdist_wheel for resampy ... - \ done
  Stored in directory: /content/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built librosa audioread resampy


Following are the class implementation of preprocesssing audio files

In [0]:
from os import listdir
from os.path import isfile,join

# We'll need numpy for some mathematical operations
import numpy as np

# matplotlib for displaying the output
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')

# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display
import math

class Collector:
  
  def __init__(self,pth,duration):
    

    self.myPath=pth
    self.iter=0
    self.liter=0
    self.duration=duration
    self.input_length=44100*self.duration
    
    file = open('train.csv','r')
    self.vocab=[]
    words=set()
    j=0
    for i in file:
      if(j!=0):
        #print(i)
        ls=i.split(",")[1]
        words.add(ls)
      j=j+1
    #print(words)
    #print(len(words))
    self.all_words=sorted(list(words))
    self.vocab={ self.all_words[j]:j for j in range(0,len(self.all_words)) }
    print(self.vocab)
    file.close()
    
  def pad(self,a):
    if(a.shape[1]>1400):
        return a[:,0:1400]
    else:
        return np.pad(a,[(0,0),(0,(1400-a.shape[1]))],'constant',constant_values=(0))

    
  def next_batch(self,it):
    onlyfiles = [f for f in sorted(listdir(self.myPath)) if isfile(join(self.myPath,f))]
    j=it
    max=0
    voc=dict()
    trainer=[]
    k=self.iter
    for i in onlyfiles:
        if(k>0):
          #print(self.iter)
          k=k-1
          continue
        audio_path=self.myPath+"/"+i
        #print(i)
        #y,sr=librosa.load(audio_path, sr=44100)
        #print(y.shape)
        data,sr=librosa.load(audio_path, sr=44100)
        
        # Random offset / Padding
        if len(data) > self.input_length:
            max_offset = len(data) - self.input_length
            #print(max_offset)
            offset = np.random.randint(max_offset)
            #print(offset)
            data = data[offset:(self.input_length+offset)]
        else:
            if self.input_length > len(data):
                max_offset = self.input_length - len(data)
                offset = np.random.randint(max_offset)
            else:
                offset = 0
            data = np.pad(data, (offset, self.input_length - len(data) - offset), "constant")
        #S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
        #log_S = librosa.power_to_db(S, ref=np.max)
        mfcc=librosa.feature.mfcc(y=data, sr=sr, n_mfcc=40)
        #mfcc        = librosa.feature.mfcc(S=log_S, n_mfcc=13)

        # Let's pad on the first and second deltas while we're at it
        #delta_mfcc  = librosa.feature.delta(mfcc)
        #delta2_mfcc = librosa.feature.delta(mfcc, order=2)
        #print(log_S.shape)
        trainer.append(mfcc)
        j=j-1
        self.iter=self.iter+1
        if(j==0):
            break
     
    init=0
    label=[]
    temp=self.liter
    repeater=it
    file2 = open('train.csv','r')
    for iter in file2:
      if(init!=0):
        if(temp>0):
          temp=temp-1
          continue
        #print(iter.split(",")[0])
        name=iter.split(",")[1]
        label.append(self.vocab[name])
        repeater=repeater-1
        self.liter=self.liter+1
        if(repeater==0):
          break
      init=init+1

    label=np.array(label)
    #print(label.shape[0])
    label2=np.eye(len(self.all_words))[label]
    #print(label2.shape)
    file2.close()
    
        
    
    return np.array(trainer),label2
        #print(log_S.shape)
        #if(log_S.shape[1] in voc):
         #   voc[log_S.shape[1]]+=1
        #else:
            #voc[log_S.shape[1]]=1
        #if(log_S.shape[1]>max):
         #   max=log_S.shape[1]
    #print(max)
    #print(math.ceil(sum(sorted(voc, key=voc.get, reverse=True)[:10])/2))
    #print(math.ceil(sum(sorted(voc, key=voc.get, reverse=True)[:20])/2))
    #print(sorted(voc, key=voc.get, reverse=True)[:1])

In [374]:
coll=Collector("audio_train",2)
train,label=coll.next_batch(5)

{'Acoustic_guitar': 0, 'Applause': 1, 'Bark': 2, 'Bass_drum': 3, 'Burping_or_eructation': 4, 'Bus': 5, 'Cello': 6, 'Chime': 7, 'Clarinet': 8, 'Computer_keyboard': 9, 'Cough': 10, 'Cowbell': 11, 'Double_bass': 12, 'Drawer_open_or_close': 13, 'Electric_piano': 14, 'Fart': 15, 'Finger_snapping': 16, 'Fireworks': 17, 'Flute': 18, 'Glockenspiel': 19, 'Gong': 20, 'Gunshot_or_gunfire': 21, 'Harmonica': 22, 'Hi-hat': 23, 'Keys_jangling': 24, 'Knock': 25, 'Laughter': 26, 'Meow': 27, 'Microwave_oven': 28, 'Oboe': 29, 'Saxophone': 30, 'Scissors': 31, 'Shatter': 32, 'Snare_drum': 33, 'Squeak': 34, 'Tambourine': 35, 'Tearing': 36, 'Telephone': 37, 'Trumpet': 38, 'Violin_or_fiddle': 39, 'Writing': 40}


In [375]:
print(train.shape)

(5, 40, 173)


In [0]:
train,label=coll.next_batch(5)

In [14]:
print(train.shape)
print(label.shape)

(5, 40, 173)
(5, 41)


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Embedding,LSTM
from keras.layers import Lambda, Input
from keras.layers import Conv1D, GlobalAveragePooling3D, MaxPooling1D,Conv2D, GlobalAveragePooling2D,GlobalMaxPooling1D
from keras.optimizers import SGD,Adadelta,Adam
from keras.backend import tf as ktf
from keras.layers import (Convolution2D, GlobalAveragePooling2D, BatchNormalization, Flatten,
                          GlobalMaxPool2D, MaxPool2D, concatenate, Activation)
from keras.utils import Sequence, to_categorical

In [0]:
file = open('train.csv','r')
vocab=[]
words=set()
j=0
for i in file:
	if(j!=0):
		#print(i)
		ls=i.split(",")[1]
		words.add(ls)
	j=j+1
#print(words)
#print(len(words))
all_words=sorted(list(words))
vocab={ all_words[j]:j for j in range(0,len(all_words)) }
print(vocab)
file.close()
k=0
label=[]
file2 = open('train.csv','r')
for iter in file2:
	if(k!=0):
		name=iter.split(",")[1]
		label.append(vocab[name])
	k=k+1
	if(k==1001):
		break

label=np.array(label)
print(label.shape[0])
label2=np.eye(len(all_words))[label]
print(label2.shape)

{'Acoustic_guitar': 0, 'Applause': 1, 'Bark': 2, 'Bass_drum': 3, 'Burping_or_eructation': 4, 'Bus': 5, 'Cello': 6, 'Chime': 7, 'Clarinet': 8, 'Computer_keyboard': 9, 'Cough': 10, 'Cowbell': 11, 'Double_bass': 12, 'Drawer_open_or_close': 13, 'Electric_piano': 14, 'Fart': 15, 'Finger_snapping': 16, 'Fireworks': 17, 'Flute': 18, 'Glockenspiel': 19, 'Gong': 20, 'Gunshot_or_gunfire': 21, 'Harmonica': 22, 'Hi-hat': 23, 'Keys_jangling': 24, 'Knock': 25, 'Laughter': 26, 'Meow': 27, 'Microwave_oven': 28, 'Oboe': 29, 'Saxophone': 30, 'Scissors': 31, 'Shatter': 32, 'Snare_drum': 33, 'Squeak': 34, 'Tambourine': 35, 'Tearing': 36, 'Telephone': 37, 'Trumpet': 38, 'Violin_or_fiddle': 39, 'Writing': 40}
1000
(1000, 41)


In [344]:
model = Sequential()  
  
model.add(Convolution2D(32, (3,3),padding="same",input_shape=(40,173,1)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D((2,2)))

model.add(Convolution2D(32, (3,3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D((3,3)))

model.add(Convolution2D(32, (3,3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D((4,4)))

#model.add(Convolution2D(32, (3,3), padding="same"))
#model.add(BatchNormalization())
#model.add(Activation("relu"))
#model.add(MaxPool2D((4,4)))
#model.add(GlobalAveragePooling2D())
#model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dense(41, activation="softmax"))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adam=Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_121 (Conv2D)          (None, 40, 173, 32)       320       
_________________________________________________________________
batch_normalization_143 (Bat (None, 40, 173, 32)       128       
_________________________________________________________________
activation_143 (Activation)  (None, 40, 173, 32)       0         
_________________________________________________________________
max_pooling2d_122 (MaxPoolin (None, 20, 86, 32)        0         
_________________________________________________________________
conv2d_122 (Conv2D)          (None, 20, 86, 32)        9248      
_________________________________________________________________
batch_normalization_144 (Bat (None, 20, 86, 32)        128       
_________________________________________________________________
activation_144 (Activation)  (None, 20, 86, 32)        0         
__________

In [190]:
coll=Collector("audio_train",2)


{'Acoustic_guitar': 0, 'Applause': 1, 'Bark': 2, 'Bass_drum': 3, 'Burping_or_eructation': 4, 'Bus': 5, 'Cello': 6, 'Chime': 7, 'Clarinet': 8, 'Computer_keyboard': 9, 'Cough': 10, 'Cowbell': 11, 'Double_bass': 12, 'Drawer_open_or_close': 13, 'Electric_piano': 14, 'Fart': 15, 'Finger_snapping': 16, 'Fireworks': 17, 'Flute': 18, 'Glockenspiel': 19, 'Gong': 20, 'Gunshot_or_gunfire': 21, 'Harmonica': 22, 'Hi-hat': 23, 'Keys_jangling': 24, 'Knock': 25, 'Laughter': 26, 'Meow': 27, 'Microwave_oven': 28, 'Oboe': 29, 'Saxophone': 30, 'Scissors': 31, 'Shatter': 32, 'Snare_drum': 33, 'Squeak': 34, 'Tambourine': 35, 'Tearing': 36, 'Telephone': 37, 'Trumpet': 38, 'Violin_or_fiddle': 39, 'Writing': 40}


In [192]:
print(train1.shape)

(2000, 40, 173)


In [0]:
from keras.utils import to_categorical

In [0]:
trainin=np.concatenate((train1,train2),axis=0)

In [0]:
labelin=np.concatenate((label1,label2),axis=0)

In [0]:
train3,label3=coll.next_batch(2000)

In [0]:
trainin2=np.concatenate((trainin,train3),axis=0)

In [0]:
labelin2=np.concatenate((labelin,label3),axis=0)

In [0]:
trainin=np.concatenate((trainin2,train4),axis=0)

In [0]:
labelin=np.concatenate((labelin2,label4),axis=0)

In [0]:
train4,label4=coll.next_batch(2000)

In [0]:
train5,label5=coll.next_batch(2000)

In [205]:
trainin.shape

(8000, 40, 173)

In [0]:
Train=np.concatenate((trainin,train5),axis=0)

In [0]:
Label=np.concatenate((labelin,label5),axis=0)

In [443]:
print(Train.shape)   #Loaded Train and Label files of Training audio files
print(Label.shape)

(9473, 40, 173, 1)
(9473, 41)


In [0]:
Train=np.expand_dims(Train,axis=-1)

In [0]:
VTrain=Train[8000:9473]  #seperating audio files for validation

In [0]:
VLabel=Label[8000:9473]

In [0]:
Label=Label[0:8000]

In [308]:
print(Train.shape)
print(VTrain.shape)

(8000, 40, 173)
(1473, 40, 173)


In [276]:
print(Label.shape)
print(VLabel.shape)

(8000, 41)
(1473, 41)


In [0]:
Train=np.expand_dims(Train,axis=-1)

In [361]:
model.fit(Train, Label, batch_size=64, epochs=15,shuffle=True)

Epoch 1/15
9473/9473 [==============================] - 7s 748us/step - loss: 1.4959 - acc: 0.5934
Epoch 2/15
9473/9473 [==============================] - 7s 729us/step - loss: 1.4766 - acc: 0.6010
Epoch 3/15
9473/9473 [==============================] - 7s 732us/step - loss: 1.4584 - acc: 0.6057
Epoch 4/15
9473/9473 [==============================] - 7s 729us/step - loss: 1.4294 - acc: 0.6152
Epoch 5/15
9473/9473 [==============================] - 7s 731us/step - loss: 1.4115 - acc: 0.6183
Epoch 6/15
9473/9473 [==============================] - 7s 732us/step - loss: 1.3803 - acc: 0.6264
Epoch 7/15
9473/9473 [==============================] - 7s 735us/step - loss: 1.3659 - acc: 0.6343
Epoch 8/15
9473/9473 [==============================] - 7s 737us/step - loss: 1.3405 - acc: 0.6349
Epoch 9/15
9473/9473 [==============================] - 7s 736us/step - loss: 1.3342 - acc: 0.6367
Epoch 10/15
9473/9473 [==============================] - 7s 740us/step - loss: 1.3034 - acc: 0.6459
Epoch 11/

In [322]:
VTrain=np.expand_dims(VTrain,axis=-1)
model.evaluate(VTrain,VLabel,verbose=1,batch_size=64)

1473/1473 [==============================] - 2s 1ms/step


[1.8089967052384615, 0.5424304141208418]

In [236]:
model.fit(Train, Label, batch_size=32, epochs=15)

Epoch 1/15
9473/9473 [==============================] - 15s 2ms/step - loss: 2.7665 - acc: 0.2680
Epoch 2/15
9473/9473 [==============================] - 12s 1ms/step - loss: 2.1434 - acc: 0.4169
Epoch 3/15
9473/9473 [==============================] - 12s 1ms/step - loss: 1.8473 - acc: 0.4908
Epoch 4/15
9473/9473 [==============================] - 12s 1ms/step - loss: 1.6606 - acc: 0.5437
Epoch 5/15
9473/9473 [==============================] - 12s 1ms/step - loss: 1.5255 - acc: 0.5748
Epoch 6/15
9473/9473 [==============================] - 12s 1ms/step - loss: 1.4085 - acc: 0.5997
Epoch 7/15
9473/9473 [==============================] - 12s 1ms/step - loss: 1.3067 - acc: 0.6346
Epoch 8/15
9473/9473 [==============================] - 12s 1ms/step - loss: 1.2172 - acc: 0.6563
Epoch 9/15
9473/9473 [==============================] - 12s 1ms/step - loss: 1.1354 - acc: 0.6757
Epoch 10/15
9473/9473 [==============================] - 12s 1ms/step - loss: 1.0699 - acc: 0.6965
Epoch 11/15
9473/94

In [413]:
model2 = Sequential()  
  
model2.add(Convolution2D(32, (4,10),padding="same",input_shape=(40,173,1)))
model2.add(BatchNormalization())
model2.add(Activation("relu"))
model2.add(MaxPool2D())

model2.add(Convolution2D(32, (4,10), padding="same"))
model2.add(BatchNormalization())
model2.add(Activation("relu"))
model2.add(MaxPool2D())

model2.add(Convolution2D(32, (4,10), padding="same"))
model2.add(BatchNormalization())
model2.add(Activation("relu"))
model2.add(MaxPool2D())

model2.add(Convolution2D(32, (4,10), padding="same"))
model2.add(BatchNormalization())
model2.add(Activation("relu"))
model2.add(MaxPool2D())

model2.add(Flatten())
model2.add(Dense(64))
model2.add(BatchNormalization())
model2.add(Activation("relu"))
model2.add(Dense(41, activation="softmax"))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#adadelta=Adadelta()
adam=Adam()
model2.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_124 (Conv2D)          (None, 40, 173, 32)       1312      
_________________________________________________________________
batch_normalization_148 (Bat (None, 40, 173, 32)       128       
_________________________________________________________________
activation_148 (Activation)  (None, 40, 173, 32)       0         
_________________________________________________________________
max_pooling2d_125 (MaxPoolin (None, 20, 86, 32)        0         
_________________________________________________________________
conv2d_125 (Conv2D)          (None, 20, 86, 32)        40992     
_________________________________________________________________
batch_normalization_149 (Bat (None, 20, 86, 32)        128       
_________________________________________________________________
activation_149 (Activation)  (None, 20, 86, 32)        0         
__________

In [452]:
model3 = Sequential()  
  
model3.add(Convolution2D(32, (4,10),padding="same",input_shape=(40,173,1)))
model3.add(BatchNormalization())
model3.add(Activation("relu"))
model3.add(MaxPool2D())

model3.add(Convolution2D(32, (4,10), padding="same"))
model3.add(BatchNormalization())
model3.add(Activation("relu"))
model3.add(MaxPool2D())

model3.add(Convolution2D(32, (4,10), padding="same"))
model3.add(BatchNormalization())
model3.add(Activation("relu"))
model3.add(MaxPool2D())

model3.add(Convolution2D(32, (4,10), padding="same"))
model3.add(BatchNormalization())
model3.add(Activation("relu"))
model3.add(MaxPool2D())
model3.add(Reshape((64,10)))
model3.add(LSTM(128))
#model3.add(Flatten())
model3.add(Dense(64))
model3.add(BatchNormalization())
model3.add(Activation("relu"))
model3.add(Dense(41, activation="softmax"))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adadelta=Adadelta()
adam=Adam()
model3.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
print(model3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_148 (Conv2D)          (None, 40, 173, 32)       1312      
_________________________________________________________________
batch_normalization_176 (Bat (None, 40, 173, 32)       128       
_________________________________________________________________
activation_176 (Activation)  (None, 40, 173, 32)       0         
_________________________________________________________________
max_pooling2d_149 (MaxPoolin (None, 20, 86, 32)        0         
_________________________________________________________________
conv2d_149 (Conv2D)          (None, 20, 86, 32)        40992     
_________________________________________________________________
batch_normalization_177 (Bat (None, 20, 86, 32)        128       
_________________________________________________________________
activation_177 (Activation)  (None, 20, 86, 32)        0         
__________

In [0]:
from keras.layers import Reshape,LSTM

In [453]:
model3.fit(Train, Label, batch_size=32, epochs=5)

Epoch 1/5
9473/9473 [==============================] - 69s 7ms/step - loss: 3.0392 - acc: 0.1608
Epoch 2/5
9473/9473 [==============================] - 64s 7ms/step - loss: 2.5165 - acc: 0.2981
Epoch 3/5
9473/9473 [==============================] - 64s 7ms/step - loss: 2.2472 - acc: 0.3609
Epoch 4/5
9473/9473 [==============================] - 64s 7ms/step - loss: 2.0681 - acc: 0.4117
Epoch 5/5
9473/9473 [==============================] - 64s 7ms/step - loss: 1.9351 - acc: 0.4495


In [454]:
model3.fit(Train, Label, batch_size=32, epochs=10)

Epoch 1/10
9473/9473 [==============================] - 64s 7ms/step - loss: 1.8398 - acc: 0.4697
Epoch 2/10
9473/9473 [==============================] - 64s 7ms/step - loss: 1.7575 - acc: 0.4973
Epoch 3/10
9473/9473 [==============================] - 64s 7ms/step - loss: 1.6630 - acc: 0.5217
Epoch 4/10
9473/9473 [==============================] - 64s 7ms/step - loss: 1.5938 - acc: 0.5409
Epoch 5/10
9473/9473 [==============================] - 64s 7ms/step - loss: 1.5401 - acc: 0.5596
Epoch 6/10
9473/9473 [==============================] - 64s 7ms/step - loss: 1.4623 - acc: 0.5755
Epoch 7/10
9473/9473 [==============================] - 64s 7ms/step - loss: 1.4160 - acc: 0.5894
Epoch 8/10
9473/9473 [==============================] - 64s 7ms/step - loss: 1.3532 - acc: 0.6059
Epoch 9/10
9473/9473 [==============================] - 64s 7ms/step - loss: 1.2987 - acc: 0.6265
Epoch 10/10
9473/9473 [==============================] - 64s 7ms/step - loss: 1.2530 - acc: 0.6324


In [458]:
model3.fit(Train, Label, batch_size=32, epochs=5)

Epoch 1/5
9473/9473 [==============================] - 63s 7ms/step - loss: 1.2160 - acc: 0.6467
Epoch 2/5
9473/9473 [==============================] - 64s 7ms/step - loss: 1.1792 - acc: 0.6545
Epoch 3/5
9473/9473 [==============================] - 64s 7ms/step - loss: 1.1462 - acc: 0.6629
Epoch 4/5
9473/9473 [==============================] - 64s 7ms/step - loss: 1.0952 - acc: 0.6796
Epoch 5/5
9473/9473 [==============================] - 64s 7ms/step - loss: 1.0588 - acc: 0.6913


In [469]:
model3.fit(Train, Label, batch_size=32, epochs=2)

Epoch 1/2
9473/9473 [==============================] - 63s 7ms/step - loss: 0.6321 - acc: 0.8086
Epoch 2/2
9473/9473 [==============================] - 64s 7ms/step - loss: 0.6138 - acc: 0.8090


In [415]:
model2.evaluate(Test,tlabel,batch_size=32,verbose=1)

1600/1600 [==============================] - 3s 2ms/step


[1.6709439301490783, 0.6125]

1299/1299 [==============================] - 1s 674us/step


[1.7349350990195564, 0.5935334871602664]

In [471]:
model3.evaluate(Private_Test,plabel,batch_size=32,verbose=1)

1299/1299 [==============================] - 2s 2ms/step


[1.8956046548598173, 0.5696689757684087]

In [456]:
model2.evaluate(Public_Test,pulabel,batch_size=32,verbose=1)

301/301 [==============================] - 0s 785us/step


[1.404808348595502, 0.6478405325515721]

In [468]:
model3.evaluate(Public_Test,pulabel,batch_size=32,verbose=1)

301/301 [==============================] - 1s 2ms/step


[1.5147284154480083, 0.6013289039515182]

In [151]:
model3.fit(Train, Label, batch_size=32, epochs=2

Epoch 1/2
9463/9463 [==============================] - 11s 1ms/step - loss: 3.5342 - acc: 0.0372
Epoch 2/2
9463/9463 [==============================] - 11s 1ms/step - loss: 3.4402 - acc: 0.0396


In [0]:
def myGenerator():
        coll=Collector("audio_train")
        while 1:
          for i in range(94):
              x_train,y_train=coll.next_batch(100)
              print(y_train.shape)
              yield x_train,y_train

In [0]:
model.fit_generator(myGenerator(),steps_per_epoch=94, nb_epoch = 1, verbose=2, callbacks=[], validation_data=None, class_weight=None, nb_worker=1)  #my old method to deal big data..doesnt work..has some issues

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=94, verbose=2, callbacks=[], validation_data=None, class_weight=None, epochs=1, workers=1)`
  """Entry point for launching an IPython kernel.


Epoch 1/1{'Acoustic_guitar': 0, 'Applause': 1, 'Bark': 2, 'Bass_drum': 3, 'Burping_or_eructation': 4, 'Bus': 5, 'Cello': 6, 'Chime': 7, 'Clarinet': 8, 'Computer_keyboard': 9, 'Cough': 10, 'Cowbell': 11, 'Double_bass': 12, 'Drawer_open_or_close': 13, 'Electric_piano': 14, 'Fart': 15, 'Finger_snapping': 16, 'Fireworks': 17, 'Flute': 18, 'Glockenspiel': 19, 'Gong': 20, 'Gunshot_or_gunfire': 21, 'Harmonica': 22, 'Hi-hat': 23, 'Keys_jangling': 24, 'Knock': 25, 'Laughter': 26, 'Meow': 27, 'Microwave_oven': 28, 'Oboe': 29, 'Saxophone': 30, 'Scissors': 31, 'Shatter': 32, 'Snare_drum': 33, 'Squeak': 34, 'Tambourine': 35, 'Tearing': 36, 'Telephone': 37, 'Trumpet': 38, 'Violin_or_fiddle': 39, 'Writing': 40}

(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(100, 41)
(10

Class to generate the Test Files

In [0]:
#lister=dict()
class Collector2:
  
  def __init__(self,pth,duration):
    

    self.myPath=pth
    self.iter=0
    self.liter=0
    self.duration=duration
    self.input_length=44100*self.duration
    self.lister=dict()   
  def pad(self,a):
    if(a.shape[1]>1400):
        return a[:,0:1400]
    else:
        return np.pad(a,[(0,0),(0,(1400-a.shape[1]))],'constant',constant_values=(0))

    
  def next_batch(self,it):
    onlyfiles = [f for f in sorted(listdir(self.myPath)) if isfile(join(self.myPath,f))]
    j=it
    max=0
    voc=dict()
    trainer=[]
    k=self.iter
    for i in onlyfiles:
        if(k>0):
          #print(self.iter)
          k=k-1
          continue
        audio_path=self.myPath+"/"+i
        #print(i)
        data,sr=librosa.load(audio_path, sr=44100)
        if(data.shape[0]==0):
          self.lister[i]='None'
          continue
        #print(y.shape)
        self.lister[i]=''
        #data,sr=librosa.load(audio_path, sr=44100)
        
        # Random offset / Padding
        if len(data) > self.input_length:
            max_offset = len(data) - self.input_length
            #print(max_offset)
            offset = np.random.randint(max_offset)
            #print(offset)
            data = data[offset:(self.input_length+offset)]
        else:
            if self.input_length > len(data):
                max_offset = self.input_length - len(data)
                offset = np.random.randint(max_offset)
            else:
                offset = 0
            data = np.pad(data, (offset, self.input_length - len(data) - offset), "constant")
        #S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
        #log_S = librosa.power_to_db(S, ref=np.max)
        mfcc=librosa.feature.mfcc(y=data, sr=sr, n_mfcc=40)
        #S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
        #log_S = librosa.power_to_db(S, ref=np.max)
        
        #mfcc        = librosa.feature.mfcc(S=log_S, n_mfcc=13)

        # Let's pad on the first and second deltas while we're at it
        #delta_mfcc  = librosa.feature.delta(mfcc)
        #delta2_mfcc = librosa.feature.delta(mfcc, order=2)
        #print(log_S.shape)
        trainer.append(mfcc)
        j=j-1
        self.iter=self.iter+1
        if(j==0):
            break
     
    return np.array(trainer),self.lister
        #print(log_S.shape)
        #if(log_S.shape[1] in voc):
         #   voc[log_S.shape[1]]+=1
        #else:
            #voc[log_S.shape[1]]=1
        #if(log_S.shape[1]>max):
         #   max=log_S.shape[1]
    #print(max)
    #print(math.ceil(sum(sorted(voc, key=voc.get, reverse=True)[:10])/2))
    #print(math.ceil(sum(sorted(voc, key=voc.get, reverse=True)[:20])/2))
    #print(sorted(voc, key=voc.get, reverse=True)[:1])

In [0]:
coll2=Collector2("audio_test",2)

In [0]:
X_test1,lister=coll2.next_batch(3000)

In [0]:
X_test2,lister=coll2.next_batch(3000)

In [0]:
X_test3,lister=coll2.next_batch(3000)

In [0]:
X_test4,lister=coll2.next_batch(1000)

In [220]:
X_test4.shape

(400, 40, 173)

In [0]:
X_testin=np.concatenate((X_test1,X_test2),axis=0)

In [0]:
X_testin2=np.concatenate((X_test3,X_test4),axis=0)

In [0]:
X_test=np.concatenate((X_testin,X_testin2),axis=0)

In [0]:
X_test=np.concatenate((X_test,X_test4),axis=0)

In [0]:
X_test=np.expand_dims(X_test,axis=-1)

In [350]:
print(X_test.shape)
print(len(lister))

(9400, 40, 173, 1)
9400


In [0]:
X_test=np.expand_dims(X_test,axis=-1)

In [0]:
la=model.predict_classes(X_test,batch_size=64)
lb=model.predict(X_test,batch_size=64)

In [353]:
print(la)

[35 20 25 ...  6  3 20]


In [0]:
#(-lb).argsort()[:,:3]
tb=(-lb).argsort()[:,:3]


In [0]:
file = open('train.csv','r')
words2=set()
j=0
for i in file:
  if(j!=0):
    #print(i)
    ls=i.split(",")[1]
    words2.add(ls)
  j=j+1
#print(words)
#print(len(words))
all_words2=sorted(list(words2))

In [356]:
print(all_words2)
print(tb.shape)
print(tb[0])
print(all_words2[int(tb[0][0])],all_words2[int(tb[0][1])],all_words2[int(tb[0][2])])
pred=all_words2[int(tb[0][0])]+" "+all_words2[int(tb[0][1])]+" "+all_words2[int(tb[0][2])]
print(pred)

['Acoustic_guitar', 'Applause', 'Bark', 'Bass_drum', 'Burping_or_eructation', 'Bus', 'Cello', 'Chime', 'Clarinet', 'Computer_keyboard', 'Cough', 'Cowbell', 'Double_bass', 'Drawer_open_or_close', 'Electric_piano', 'Fart', 'Finger_snapping', 'Fireworks', 'Flute', 'Glockenspiel', 'Gong', 'Gunshot_or_gunfire', 'Harmonica', 'Hi-hat', 'Keys_jangling', 'Knock', 'Laughter', 'Meow', 'Microwave_oven', 'Oboe', 'Saxophone', 'Scissors', 'Shatter', 'Snare_drum', 'Squeak', 'Tambourine', 'Tearing', 'Telephone', 'Trumpet', 'Violin_or_fiddle', 'Writing']
(9400, 3)
[35 16 32]
Tambourine Finger_snapping Shatter
Tambourine Finger_snapping Shatter


In [0]:
ks=0
myData=[]
myData.append(['fname','label'])
for i in lister:
  a=[]
  if(lister[i]==''):
      pred=all_words2[int(tb[ks][0])]+" "+all_words2[int(tb[ks][1])]+" "+all_words2[int(tb[ks][2])]
      #pred=all_words2[int(tb[ks][0])]
      ks=ks+1
      a.append(i)
      a.append(pred)
  else:
      a.append(i)
      a.append('None')
  myData.append(a)
    

In [358]:
myData

[['fname', 'label'],
 ['00063640.wav', 'Tambourine Finger_snapping Shatter'],
 ['0013a1db.wav', 'Gong Acoustic_guitar Meow'],
 ['002bb878.wav', 'Knock Bass_drum Gunshot_or_gunfire'],
 ['002d392d.wav', 'Bass_drum Knock Double_bass'],
 ['00326aa9.wav', 'Oboe Saxophone Clarinet'],
 ['0038a046.wav', 'Knock Electric_piano Gong'],
 ['003995fa.wav', 'Laughter Squeak Chime'],
 ['005ae625.wav', 'Acoustic_guitar Flute Gong'],
 ['007759c4.wav', 'Flute Clarinet Cello'],
 ['008afd93.wav', 'Violin_or_fiddle Saxophone Cello'],
 ['00a161c0.wav', 'Cello Acoustic_guitar Double_bass'],
 ['00a7a2f6.wav', 'Clarinet Flute Gong'],
 ['00ae03f6.wav', 'Squeak Chime Fireworks'],
 ['00b2404e.wav', 'Cello Violin_or_fiddle Acoustic_guitar'],
 ['00beb030.wav', 'Clarinet Cello Chime'],
 ['00c4d5b8.wav', 'Acoustic_guitar Gong Cello'],
 ['00c92c05.wav', 'Acoustic_guitar Gong Flute'],
 ['00ccf065.wav', 'Bass_drum Snare_drum Knock'],
 ['00d0ab77.wav', 'Clarinet Gong Cello'],
 ['00dffe3a.wav', 'Bass_drum Snare_drum Gunsho

In [359]:
print(len(myData))
import csv
file = open('Tes.csv','w')
with file:
	writer=csv.writer(file)
	writer.writerows(myData)

9401


In [360]:
!kaggle competitions submit -c freesound-audio-tagging -f Tes.csv -m "valid epochs"

Successfully submitted to Freesound General-Purpose Audio Tagging Challenge

In [0]:
from google.colab import files
files.download('Tes.csv')

Following code is to upload csv files to google drive

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [260]:
uploaded = drive.CreateFile({'title': 'Trainp.csv'})
uploaded.SetContentFile('Trainp.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1eJnImYoFMWkw7klAkIRsLBi7ti9s18QS
